# 豆瓣电影的多方法解析
https://movie.douban.com/tag/Top100

### 1. 分析网页，确认爬取目标的数据类型。
   - #### 打开 [目标url](https://movie.douban.com/tag/Top100), 定位数据位置
   ![爬取源网址](./images/index.png)  
   - #### 定位需要的数据位置，查看爬取目标。  
   ![爬取目标](./images/data_target.png)  
   由图可得，我们需要的数据分别为，['海报', '电影名', '上映日期', '演员', '评分', '评价人数']
   - #### 查看请求，分析数据来源请求（F12 >> network 打开请求界面，如下图）
   ![抓包准备](./images/capture_package_ready.png)  
   - #### 确认数据请求来源(Ctrl + F 搜索: 辛德勒)
   ![抓包准备](./images/confirm_request.png)  
   上图可知，该请求只有一个，所以就能轻松的确定来源拉！ 
   - #### 查看headers，分析请求报文
   ![请求分析](./images/requests_ann.png)  
   分析结果如图，所以我们可以得出以下结论：  
   
    
| 信息     | 结果                                  |
| -------- | -------------------------------------|
| 请求地址 | <https://movie.douban.com/tag/Top100> |
| 请求方法 | Get                                   |
| 响应格式 | text 文本                             |
| 编码     | UTF-8                                |

## 2. 利用requests进行请求测试
`requests.get`  
定义请求函数，`get_data`  
返回`text`数据  

In [1]:
import requests
from requests.exceptions import HTTPError

In [2]:
def get_data(url):
    
    response = requests.get(url)
    if response.status_code == requests.codes.ok:  # 检测状态码
        return response.text
    else:
        response.raise_for_status()  # 4xx 5xx 时,引出错误 代替 raise requests.exception.HTTPError

In [3]:
url = "https://movie.douban.com/tag/Top100"

In [4]:
data = get_data(url)  # 获取数据

## 3. 提取数据
   - 正则提取
   - BeautifulSoup 提取
   - Xpath 提取
   - scrapy 混合提取

### 1. 正则提取
- 观察数据位置
![电影名，海报地址](./images/re_name_poster.png)

In [5]:
import re

### 提取 海报地址以及电影名称
**通过查看该请求的响应内容快速进行复制匹配,如下图搜索:**
![复制匹配](./images/re_get_data_1.png)  
用到的匹配规则提示:
   - "." 表示任意非空格换行等字符
   - ".*?"  表示贪婪匹配,最少匹配一次
   - "()"  表示提取()中的内容
   -  "\\w" 表示正常字符,比如英文字母,中文等常见文字
   - ".+"  表示至少匹配一次任意字符

In [40]:
# 设置提取表达式 
poster_pattern = re.compile(r"""<a class="nbg" href=".*?"  title=".*?">.*?<img src="(.*?)" width="75" alt="(.*?)" class=""/>.*?</a>""", re.S)  # 海报的正则表达式
movie_name_pattern = re.compile(r""" <div class="pl2">.*? <a href=".*?"  class="">.*?(\w+).*?<span style="font-size:13px;">(.*?)</span>.*?</a>""", re.S)  # 电影名正则表达式
poster_res = re.findall(poster_pattern, data)
movie_name_res = re.findall(movie_name_pattern, data)
poster_res, movie_name_res

([('https://img3.doubanio.com/view/photo/s_ratio_poster/public/p492406163.jpg',
   '辛德勒的名单'),
  ('https://img1.doubanio.com/view/photo/s_ratio_poster/public/p1546987967.jpg',
   '狩猎'),
  ('https://img1.doubanio.com/view/photo/s_ratio_poster/public/p477229647.jpg',
   '美国往事'),
  ('https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2173577632.jpg',
   '十二怒汉'),
  ('https://img1.doubanio.com/view/photo/s_ratio_poster/public/p1808872109.jpg',
   '窃听风暴'),
  ('https://img3.doubanio.com/view/photo/s_ratio_poster/public/p1910901025.jpg',
   '天堂电影院'),
  ('https://img3.doubanio.com/view/photo/s_ratio_poster/public/p616779645.jpg',
   '教父'),
  ('https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2119675128.jpg',
   '一一'),
  ('https://img1.doubanio.com/view/photo/s_ratio_poster/public/p792238287.jpg',
   '飞越疯人院'),
  ('https://img3.doubanio.com/view/photo/s_ratio_poster/public/p1181775734.jpg',
   '鬼子来了'),
  ('https://img1.doubanio.com/view/photo/s_ratio_poster/public/p792443418.jp

**查看结果好像没什么问题, 我们用长度比较来看看数量是否一致**

In [41]:
len(poster_res) == len(movie_name_res)

True

**长度一致,看来匹配规则在这里没问题  
我们用同样的方法,提取其他需要的数据**  
***这里有个小技巧,用额外的字段来得到唯一匹配, 如下图:***  
需要的数据:  
![需要的数据](./images/unique1.png)   
额外数据匹配:  
![额外匹配](./images/unique2.png)

In [43]:
#### 匹配上映日期和演员
time_performer_pattern = re.compile("""</span>.*?<p class="pl">(.*?/?.*?)/(.*?)</p>.*?<div class="star clearfix">""", re.S)
re.findall(time_performer_pattern, data)

[('1993-11-30(华盛顿首映) / 1994-02-04(美国) ',
  ' 连姆·尼森 / 本·金斯利 / 拉尔夫·费因斯 / 卡罗琳·古多尔 / 乔纳森·萨加尔 / 艾伯丝·戴维兹 / 马尔戈萨·格贝尔 / 马克·伊瓦涅 / 碧翠斯·马科拉 / 安德烈·瑟韦林 / 弗里德里希·冯·图恩 / 克齐斯茨托夫·拉夫特...'),
 ('2012-05-20(戛纳电影节) / 2013-01-10(丹麦) ',
  ' 麦斯·米科尔森 / 托玛斯·博·拉森 / 安妮卡·韦德科普 / 拉丝·弗格斯托姆 / 苏西·沃德 / 安妮·路易丝·哈辛 / 拉斯·兰特 / 亚历山德拉·拉帕波特 / 拉斯穆斯·林德·鲁宾 / 丹麦 / 瑞典 / 托马斯·温特伯格...'),
 ('1984-02-17(波士顿首映) / 1984-09-28(意大利) ',
  ' 罗伯特·德尼罗 / 詹姆斯·伍兹 / 伊丽莎白·麦戈文 / 乔·佩西 / 波特·杨 / 塔斯黛·韦尔德 / 特里特·威廉斯 / 丹尼·爱罗 / 理查德·布赖特 / 詹姆斯·海登 / 威廉·弗西斯 / 达兰妮·弗鲁格 / 拉里·拉普...'),
 ('1957-04-13(美国) / 亨利·方达 ',
  ' 马丁·鲍尔萨姆 / 约翰·菲德勒 / 李·科布 / E.G.马绍尔 / 杰克·克卢格曼  / 爱德华·宾斯 / 杰克·瓦尔登 / 约瑟夫·史威尼 / 埃德·贝格利 / 乔治·沃斯科维奇 / 罗伯特·韦伯 / 美国 / 西德尼·吕美特 / 96 分钟...'),
 ('2006-03-23(德国) / 乌尔里希·穆埃 ',
  ' 马蒂娜·格德克 / 塞巴斯蒂安·科赫 / 乌尔里希·图库尔 / 托马斯·蒂梅 / 汉斯-尤韦·鲍尔 / 沃克马·克莱纳特 / 马提亚斯·布伦纳 / 查理·哈纳 / 赫伯特·克瑙普 / 巴斯蒂安·特罗斯特 / 玛丽·格鲁伯...'),
 ('1988-11-17(意大利) / 安东内拉·阿蒂利 ',
  ' 恩佐·卡拉瓦勒 / 艾萨·丹尼埃利 / 里奥·故罗塔 / 马克·莱昂纳蒂 / 普佩拉·玛奇奥 / 阿格妮丝·那诺 / 莱奥波多·特里耶斯泰 / 萨瓦特利·卡西欧 / 尼古拉·迪·平托 / 罗伯塔·蕾娜 / 尼诺·戴罗佐...'),
 ('1972-03-15(纽约首映) / 197